In [ ]:
# ============================================================
# 0. SETUP BÀSIC – PER EXECUTAR A GOOGLE COLAB (VERSIÓ 3 - ROBUSTA)
# ============================================================

!pip install -q sentence-transformers lightgbm

import os
import ast
import re
import numpy as np
import pandas as pd

from sentence_transformers import SentenceTransformer
from sklearn.decomposition import PCA
from sklearn.model_selection import train_test_split
import lightgbm as lgb

from collections import defaultdict
import warnings
warnings.filterwarnings("ignore")

DATA_DIR = "/content/"
train_path = os.path.join(DATA_DIR, "train.csv")
test_path  = os.path.join(DATA_DIR, "test.csv")

print("Train path:", train_path)
print("Test path:", test_path)

try:
    # Usem sep=';', forcem header=0 i mantenim 'skip' per les línies de dades dolentes
    train = pd.read_csv(train_path, sep=';', header=0, on_bad_lines='skip')
    test  = pd.read_csv(test_path,  sep=';', header=0, on_bad_lines='skip')

except Exception as e:
    print(f"Error crític durant la càrrega: {e}")
    print("Provant amb 'python' engine (més lent però més tolerant)...")
    train = pd.read_csv(train_path, sep=';', header=0, engine="python", on_bad_lines="skip")
    test  = pd.read_csv(test_path,  sep=';', header=0, engine="python", on_bad_lines="skip")


# Neteja de columnes fantasma (les ;;;; del final)
train.dropna(axis=1, how='all', inplace=True)
test.dropna(axis=1, how='all', inplace=True)

# Normalitzem un typo que he vist:
if 'has_plus_sizes' in test.columns:
    test.rename(columns={'has_plus_sizes': 'has_plus_size'}, inplace=True)

print("Train shape (raw):", train.shape)
print("Test shape (raw):", test.shape)

# --- VERIFICACIÓ DE COLUMNES CLAU ---
required_cols_train = ['ID', 'phase_in', 'weekly_demand', 'year']
required_cols_test = ['ID', 'phase_in']

# Comprovem Train
missing_in_train = [col for col in required_cols_train if col not in train.columns]
if 'phase_in' not in train.columns:
    print("ERROR GREU: 'phase_in' no s'ha carregat a 'train'. Columnes presents:")
    print(train.columns.tolist())
    raise KeyError("La columna 'phase_in' falta a train.csv després de la càrrega.")

# Comprovem Test
if 'phase_in' not in test.columns:
    print("ERROR GREU: 'phase_in' no s'ha carregat a 'test'. Columnes presents:")
    print(test.columns.tolist())
    raise KeyError("La columna 'phase_in' falta a test.csv després de la càrrega.")

print("Verificació de columnes (phase_in) OK.")

In [ ]:
# ============================================================
# 1. FUNCCIONS AUXILIARS
# ============================================================

def parse_image_embedding(col):
    """
    Assumeix que image_embedding és una string tipus "[0.1, 0.2, ...]".
    Si no és el cas, adapta aquesta funció.
    """
    def _parse(x):
        if isinstance(x, str):
            try:
                return np.array(ast.literal_eval(x), dtype="float32")
            except Exception:
                return np.nan
        elif isinstance(x, (list, np.ndarray)):
            return np.array(x, dtype="float32")
        else:
            return np.nan

    emb_list = col.apply(_parse).values
    # Filtrar NaNs
    valid_emb = [e for e in emb_list if isinstance(e, np.ndarray)]
    dim = valid_emb[0].shape[0]
    # Omplir NaNs amb zeros
    fixed = []
    for e in emb_list:
        if not isinstance(e, np.ndarray):
            fixed.append(np.zeros(dim, dtype="float32"))
        else:
            fixed.append(e)
    return np.vstack(fixed)


def build_text_description(row):
    """
    Crea una descripció textual del producte a partir d'atributs rellevants (VERSIÓ CORREGIDA).
    """
    parts = [
        str(row.get("aggregated_family", "")),
        str(row.get("family", "")),
        str(row.get("category", "")),
        str(row.get("silhouette_type", "")),
        str(row.get("length_type", "")),
        str(row.get("waist_type", "")),
        str(row.get("sleeve_length_type", "")),
        str(row.get("neck_lapel_type", "")), # Afegida
        str(row.get("fabric", "")),
        str(row.get("print_type", "")),
        str(row.get("color_name", "")),
        str(row.get("moment", "")),
        # str(row.get("ocassion", "")), # Eliminada
        str(row.get("archetype", "")),
    ]
    parts = [p for p in parts if p and p.lower() != "nan"]
    return " ".join(parts)


def add_year_if_missing(df):
    """
    Si no hi ha 'year' però sí 'id_season', pots intentar extreure l'any.
    Si ja tens 'year', no fa res.
    """
    if "year" not in df.columns:
        if "id_season" in df.columns:
            # EXEMPLE: si id_season porta l'any dins o és un codi que permet obtenir-lo.
            # Aquí deixem un placeholder. Adapta a la realitat.
            # Ara mateix no fa res intel·ligent:
            df["year"] = df["id_season"]
        else:
            raise ValueError("No 'year' nor 'id_season' in dataframe.")
    return df


In [ ]:
# ============================================================
# 2. CREAR 'year' DES DE 'phase_in' I PREPARAR TARGET D_TOTAL
# ============================================================

def create_year_from_phase_in(df):
    # Convertim phase_in a string i extreiem els 4 dígits de l'any
    # Assegurem que és string abans d'aplicar .str
    df['phase_in_str'] = df['phase_in'].astype(str)

    # Extreiem l'any (YYYY) de formats com DD/MM/YYYY
    df['year'] = df['phase_in_str'].str.extract(r'(\d{4})')

    # Convertim a numèric i omplim si hi ha algun error
    df['year'] = pd.to_numeric(df['year'], errors='coerce')

    # Si algun falla, omplim amb la moda (l'any més comú)
    if df['year'].isnull().any():
        mode_year = df['year'].mode()[0]
        df['year'] = df['year'].fillna(mode_year)

    df['year'] = df['year'].astype(int)
    df = df.drop(columns=['phase_in_str'])
    return df

# Apliquem la nova funció
train = create_year_from_phase_in(train)
test  = create_year_from_phase_in(test)

print(f"Anys detectats a Train: {train['year'].unique()}")
print(f"Anys detectats a Test: {test['year'].unique()}")


# Assegurem tipus numèric on toca
train["weekly_demand"] = pd.to_numeric(train["weekly_demand"], errors="coerce").fillna(0)
train["weekly_sales"]  = pd.to_numeric(train["weekly_sales"], errors="coerce").fillna(0)

# Agreguem a nivell ID (target D_total i info bàsica)
agg_train = (
    train
    .groupby("ID")
    .agg(
        D_total=("weekly_demand", "sum"),   # TARGET
        sales_total=("weekly_sales", "sum"),
        first_week=("num_week_iso", "min"),
        last_week=("num_week_iso", "max")
    )
    .reset_index()
)
agg_train["life_cycle_real"] = agg_train["last_week"] - agg_train["first_week"] + 1

print("Unique IDs in train:", agg_train.shape[0])

In [ ]:
# ============================================================
# 3. EXTRACCIÓ D’ATRIBUTS ESTÀTICS (UNA FILA PER ID) - CORREGIT
# ============================================================

static_cols = [
    "ID", "id_season", "aggregated_family", "family", "category", "fabric",
    "color_name", "color_rgb", "length_type", "silhouette_type", "waist_type",
    "sleeve_length_type", "heel_shape_type", "toecap_type", "woven_structure",
    "knit_structure", "print_type", "archetype", "moment", "ocassion",
    "phase_in", "phase_out", "life_cycle_length",
    "num_stores", "num_sizes", "has_plus_size", "price", "year",
    "image_embedding"
]

# No totes les columnes poden existir en tots els datasets; filtrem
static_cols_train = [c for c in static_cols if c in train.columns]
static_cols_test = [c for c in static_cols if c in test.columns]

# --- LÒGICA PER STATIC_TRAIN (Té dades setmanals) ---
# Treiem 'ID' de la llista de columnes a seleccionar, perquè ja és l'índex del groupby
cols_to_agg_train = [c for c in static_cols_train if c != "ID"]

static_train = (
    train
    .sort_values(["ID", "num_week_iso"]) # <-- 'train' sí que té 'num_week_iso'
    .groupby("ID")[cols_to_agg_train]
    .first()
    .reset_index()
)

# --- NOVA LÒGICA PER STATIC_TEST (Ja és 1 fila/ID i no té 'num_week_iso') ---
# El 'test' ja ve agregat (1 fila per ID), no cal fer groupby ni sort.
# Només hem de seleccionar les columnes estàtiques que existeixin.
static_test = test[static_cols_test].copy()
# --- FI DEL CANVI ---


print("static_train shape:", static_train.shape)
print("static_test shape:", static_test.shape)

# Merge de target + estàtics per train
train_id = agg_train.merge(static_train, on="ID", how="left")

In [ ]:
# ============================================================
# 4. EMBEDDING DE TEXT
# ============================================================

train_id["text_desc"] = train_id.apply(build_text_description, axis=1)
static_test["text_desc"] = static_test.apply(build_text_description, axis=1)

print("Exemple descripció:", train_id["text_desc"].iloc[0])

text_model = SentenceTransformer("sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2")

train_text_emb = text_model.encode(
    train_id["text_desc"].tolist(),
    batch_size=128,
    show_progress_bar=True
)
test_text_emb = text_model.encode(
    static_test["text_desc"].tolist(),
    batch_size=128,
    show_progress_bar=True
)

# Reducció dimensional
pca_text = PCA(n_components=50, random_state=42)
train_text_pca = pca_text.fit_transform(train_text_emb)
test_text_pca = pca_text.transform(test_text_emb)

text_cols = [f"text_emb_{i}" for i in range(train_text_pca.shape[1])]
df_train_text = pd.DataFrame(train_text_pca, columns=text_cols, index=train_id.index)
df_test_text  = pd.DataFrame(test_text_pca,  columns=text_cols, index=static_test.index)


In [ ]:
# ============================================================
# 5. EMBEDDING D’IMATGE
# ============================================================

if "image_embedding" not in train_id.columns:
    raise ValueError("No 'image_embedding' column in train_id; adapta aquesta part.")

img_emb_train = parse_image_embedding(train_id["image_embedding"])
img_emb_test  = parse_image_embedding(static_test["image_embedding"])

print("Image embedding shape (train):", img_emb_train.shape)
print("Image embedding shape (test):", img_emb_test.shape)

# PCA per comprimir
pca_img = PCA(n_components=50, random_state=42)
train_img_pca = pca_img.fit_transform(img_emb_train)
test_img_pca  = pca_img.transform(img_emb_test)

img_cols = [f"img_emb_{i}" for i in range(train_img_pca.shape[1])]
df_train_img = pd.DataFrame(train_img_pca, columns=img_cols, index=train_id.index)
df_test_img  = pd.DataFrame(test_img_pca,  columns=img_cols, index=static_test.index)


In [ ]:
# ============================================================
# 6. FEATURES TEMPORALS AVANÇADES
#   (category, family, silhouette_type, moment) - CORREGIDA
# ============================================================

# Ens assegurem que tenim aquestes columnes al raw train
# --- 'ocassion' eliminada de la llista ---
for col in ["category", "family", "silhouette_type", "moment"]:
    if col not in train.columns:
        train[col] = "unknown" # Afegim 'unknown' si no existeix

# Helper: calcula agregats per (group_col, year) i deriva features de tendència
def build_temporal_features(df, group_col, demand_col="weekly_demand", price_col="price"):
    """
    df: dataframe setmanal (train)
    group_col: columna (str) de agrupació, ex 'category', 'family', etc.
    Retorna un dataframe amb:
      group_col, year + features:
        total_demand, mean_price, num_products,
        demand_growth, demand_trend_slope, demand_trend_vol, ...
    """
    tmp = (
        df
        .groupby([group_col, "year"])
        .agg(
            total_demand=(demand_col, "sum"),
            mean_price=(price_col, "mean"),
            num_products=("ID", "nunique")
        )
        .reset_index()
        .sort_values([group_col, "year"])
    )

    # Creixement interanual
    tmp["total_demand_prev"] = tmp.groupby(group_col)["total_demand"].shift(1)
    tmp["demand_growth"] = tmp["total_demand"] / tmp["total_demand_prev"] - 1
    tmp["demand_growth"] = tmp["demand_growth"].replace([np.inf, -np.inf], np.nan).fillna(0)

    # Ara calculem slope de la demanda per grup (usant els punts any vs total_demand)
    slopes = []
    vols   = []

    for g, sub in tmp.groupby(group_col):
        years = sub["year"].values
        dem   = sub["total_demand"].values

        if len(sub) >= 2:
            # slope via regressió lineal
            x = years.astype(float)
            y = dem.astype(float)
            # normalitzem x per estabilitat
            x_norm = x - x.mean()
            slope = np.polyfit(x_norm, y, 1)[0]
            vol = np.std(sub["demand_growth"].values)
        else:
            slope = 0.0
            vol = 0.0

        slopes.extend([slope] * len(sub))
        vols.extend([vol] * len(sub))

    tmp["demand_trend_slope"] = slopes
    tmp["demand_trend_vol"]   = vols

    # Renombrem columnes per incloure el nom del grup
    out = tmp.rename(columns={
        "total_demand": f"{group_col}_total_demand",
        "mean_price":   f"{group_col}_mean_price",
        "num_products": f"{group_col}_num_products",
        "demand_growth": f"{group_col}_demand_growth",
        "demand_trend_slope": f"{group_col}_demand_trend_slope",
        "demand_trend_vol": f"{group_col}_demand_trend_vol"
    })

    # --- CANVI CLAU AQUÍ ---
    # Esborrem la columna intermèdia 'total_demand_prev' abans de retornar
    if "total_demand_prev" in out.columns:
        out = out.drop(columns=["total_demand_prev"])
    # --- FI DEL CANVI ---

    return out


# Construïm features temporals per cada dimensió
temp_cat  = build_temporal_features(train, "category")
temp_fam  = build_temporal_features(train, "family")
temp_sil  = build_temporal_features(train, "silhouette_type")
temp_mom  = build_temporal_features(train, "moment")
# temp_oca  = build_temporal_features(train, "ocassion") # <-- Línia eliminada

# Merge a train_id i static_test
def merge_temporal_features(base_df, temp_df, group_col):
    cols_merge = [group_col, "year"]

    # Assegurem que les columnes de merge existeixen al base_df
    if group_col not in base_df.columns:
        base_df[group_col] = "unknown"

    feat_cols = [c for c in temp_df.columns if c not in cols_merge]
    return base_df.merge(temp_df[cols_merge + feat_cols],
                         on=cols_merge,
                         how="left")

train_id = merge_temporal_features(train_id, temp_cat, "category")
train_id = merge_temporal_features(train_id, temp_fam, "family")
train_id = merge_temporal_features(train_id, temp_sil, "silhouette_type")
train_id = merge_temporal_features(train_id, temp_mom, "moment")
# train_id = merge_temporal_features(train_id, temp_oca, "ocassion") # <-- Línia eliminada

static_test = merge_temporal_features(static_test, temp_cat, "category")
static_test = merge_temporal_features(static_test, temp_fam, "family")
static_test = merge_temporal_features(static_test, temp_sil, "silhouette_type")
static_test = merge_temporal_features(static_test, temp_mom, "moment")
# static_test = merge_temporal_features(static_test, temp_oca, "ocassion") # <-- Línia eliminada

# Omplim NaNs temporals amb valors neutres (LÒGICA CORREGIDA)
temporal_cols_train = [c for c in train_id.columns if "_demand_" in c or "_mean_price" in c or "_num_products" in c]
train_id[temporal_cols_train] = train_id[temporal_cols_train].fillna(0)

temporal_cols_test = [c for c in static_test.columns if "_demand_" in c or "_mean_price" in c or "_num_products" in c]
static_test[temporal_cols_test] = static_test[temporal_cols_test].fillna(0)
# --- FI DE LA CORRECCIÓ ---

print("Train_id shape after temporal merge:", train_id.shape)
print("Static_test shape after temporal merge:", static_test.shape)

In [ ]:
# ============================================================
# 7. CONSTRUCCIÓ DATASET FINAL (FEATURES COMPLETES) - CORREGIT
# ============================================================

# Afegim embeddings
train_feat = pd.concat([train_id, df_train_text, df_train_img], axis=1)
test_feat  = pd.concat([static_test, df_test_text, df_test_img], axis=1)

# --- NOU CANVI: Conversió manual de 'has_plus_size' ---
# Convertim la columna 'has_plus_size' (que és string "true"/"false") a integer (1/0)
# Ho fem abans de 'get_dummies' per evitar que entri com a 'object' a LightGBM

map_dict = {'true': 1, 'false': 0}

if "has_plus_size" in train_feat.columns:
    # Usem .map per convertir. Els valors no trobats (NaNs) es queden com NaN
    train_feat['has_plus_size'] = train_feat['has_plus_size'].map(map_dict)
    # Omplim els NaNs que hagin quedat (o que ja hi eren) amb 0 (valor neutre)
    train_feat['has_plus_size'] = train_feat['has_plus_size'].fillna(0).astype(int)

if "has_plus_size" in test_feat.columns:
    test_feat['has_plus_size'] = test_feat['has_plus_size'].map(map_dict)
    test_feat['has_plus_size'] = test_feat['has_plus_size'].fillna(0).astype(int)

# --- FI DEL CANVI ---


# Guardem target i IDs
y = train_feat["D_total"].values
train_ids = train_feat["ID"].values
test_ids  = test_feat["ID"].values

# Treure columnes que no volem com a input
drop_cols = [
    "D_total", "sales_total", "first_week", "last_week", "life_cycle_real",
    "text_desc", "image_embedding",
    "weekly_sales", "weekly_demand", "Production"
]

for c in drop_cols:
    if c in train_feat.columns:
        train_feat = train_feat.drop(columns=[c], errors='ignore') # Afegit errors='ignore'
    if c in test_feat.columns:
        test_feat = test_feat.drop(columns=[c], errors='ignore') # Afegit errors='ignore'

# ID no entra com a feature
if "ID" in train_feat.columns:
    train_feat = train_feat.drop(columns=["ID"])
if "ID" in test_feat.columns:
    test_feat = test_feat.drop(columns=["ID"])

# Separem tipus objecte per fer dummies
cat_cols = train_feat.select_dtypes(include=["object"]).columns.tolist()

full = pd.concat([train_feat, test_feat], axis=0)

# Dummies ràpids
full = pd.get_dummies(full, columns=cat_cols, dummy_na=True)

train_X = full.iloc[:len(train_feat)].reset_index(drop=True)
test_X  = full.iloc[len(train_feat):].reset_index(drop=True)

# --- NOU CANVI: Netejar els noms de les columnes per LightGBM ---
# Reemplacem qualsevol caràcter que no sigui lletra, número o _ per un _
import re
train_X.columns = [re.sub(r'[^A-Za-z0-9_]+', '_', col) for col in train_X.columns]
test_X.columns = [re.sub(r'[^A-Za-z0-9_]+', '_', col) for col in test_X.columns]
# --- FI DEL CANVI ---

print("train_X shape:", train_X.shape)
print("test_X shape:", train_X.shape)

In [ ]:
# ============================================================
# 8. SPLIT TEMPORAL TRAIN / VALID
# ============================================================

# Recuperem 'year' per fer split temporal (abans de treure-la de features)
# Ho traiem de train_id (mateix ordre que y)
train_years = train_id["year"].values

last_year = np.max(train_years)
print("Últim any en train:", last_year)

is_valid = train_years == last_year

X_train = train_X[~is_valid]
X_valid = train_X[is_valid]
y_train = y[~is_valid]
y_valid = y[is_valid]

print("Train size:", X_train.shape, "Valid size:", X_valid.shape)


In [ ]:
# ============================================================
# 9. LIGHTGBM QUANTÍLIC – ENTRENAR DIVERSOS QUANTILS (CORREGIT)
# ============================================================

def train_lgb_quantile(X_tr, y_tr, X_va, y_va, alpha):
    params = {
        "objective": "quantile",
        "alpha": alpha,
        "learning_rate": 0.05,
        "num_leaves": 63,
        "min_data_in_leaf": 80,
        "feature_fraction": 0.8,
        "bagging_fraction": 0.8,
        "bagging_freq": 1,
        "metric": "quantile",
        "verbose": -1,
        "seed": 42,
    }

    lgb_train = lgb.Dataset(X_tr, label=y_tr)
    lgb_valid = lgb.Dataset(X_va, label=y_va)

    # --- CANVI CLAU AQUÍ ---
    # 'early_stopping_rounds' i 'verbose_eval' ara van dins de 'callbacks'
    model = lgb.train(
        params,
        lgb_train,
        valid_sets=[lgb_train, lgb_valid],
        num_boost_round=3000,
        callbacks=[
            lgb.early_stopping(stopping_rounds=150, verbose=False), # Atura si no millora en 150 rondes
            lgb.log_evaluation(period=200) # Mostra el log cada 200 rondes
        ]
    )
    # --- FI DEL CANVI ---

    return model

quantiles = [0.5, 0.7, 0.75, 0.8, 0.85, 0.9]
models = {}
pred_valid = {}

for q in quantiles:
    print(f"\n========== Entrenant model per quantil {q} ==========")
    m = train_lgb_quantile(X_train, y_train, X_valid, y_valid, alpha=q)
    models[q] = m

    # Assegurem que 'best_iteration' té un valor
    best_iter = m.best_iteration if m.best_iteration is not None else 3000
    pred_valid[q] = m.predict(X_valid, num_iteration=best_iter)

In [ ]:
# ============================================================
# 10. CÀLCUL DEL VAR PER CADA QUANTIL I SELECCIÓ DEL MILLOR
# ============================================================

valid_demand_real = y_valid  # D_total real a validació

var_scores = {}

for q in quantiles:
    prod_q = np.maximum(pred_valid[q], 1)  # evitar 0
    full_price_sales_q = np.minimum(valid_demand_real, prod_q)
    var_q = (full_price_sales_q / prod_q).mean()
    var_scores[q] = var_q

print("VAR per quantil:")
for q, v in var_scores.items():
    print(f"  Quantil {q}: VAR = {v:.4f}")

best_q = max(var_scores, key=var_scores.get)
print("\n>>> MILLOR QUANTIL:", best_q, "amb VAR:", var_scores[best_q])


In [ ]:
# ============================================================
# 11. PREDICCIÓ EN TEST I SUBMISSION
# ============================================================

best_model = models[best_q]

test_pred = best_model.predict(test_X, num_iteration=best_model.best_iteration)
test_prod = np.maximum(test_pred, 1).astype(int)  # producció mínima 1 unitat

submission = pd.DataFrame({
    "ID": test_ids,
    "Production": test_prod
})

out_path = os.path.join(DATA_DIR, "submission.csv")
submission.to_csv(out_path, index=False)

print("Submission guardada a:", out_path)
submission.head()
